In [2]:
import utils as Utils

#ライブラリ読み込み
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold,cross_val_score
from matplotlib import pyplot as plt
#matplotlib inline
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

In [3]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [4]:
# フラグを立てる
train['flg'] = 1
test['flg'] = 0
# trainデータとtestデータを結合
all_data = pd.concat([train,test], axis=0, sort=False)

In [5]:
all_data['Attrition']=all_data['Attrition'].fillna(0)

In [6]:
# trainを変換
all_data["BusinessTravel_E"] = Utils.label_encoding(all_data["BusinessTravel"])
all_data["Department_E"] = Utils.label_encoding(all_data["Department"])
all_data["EducationField_E"] = Utils.label_encoding(all_data["EducationField"])
all_data["Gender_E"] = Utils.label_encoding(all_data["Gender"])
all_data["JobRole_E"] = Utils.label_encoding(all_data["JobRole"])
all_data["MaritalStatus_E"] = Utils.label_encoding(all_data["MaritalStatus"])
all_data["Over18_E"] = Utils.label_encoding(all_data["Over18"])
all_data["OverTime_E"] = Utils.label_encoding(all_data["OverTime"])

# all_data["Age_Binned"] = Utils.data_binned(all_data["Age"], [16, 25, 30, 35, 40, 45, 50, 55, 60], [0, 1, 2, 3, 4, 5, 6, 7])

all_data['Attrition'].value_counts()

all_data = all_data.drop(['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime'], axis=1)

In [6]:
all_data

,id,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,Attrition,flg,BusinessTravel_E,Department_E,EducationField_E,Gender_E,JobRole_E,MaritalStatus_E,Over18_E,OverTime_E
0,0,26,450.941476,7.601074,3,4,43,2,1,2,...,0.0,1,2,1,3,1,2,2,0,0
1,3,47,730.235896,26.739489,3,2,98,2,1,1,...,0.0,1,2,1,5,1,6,0,0,1
2,7,26,1082.560066,7.374739,3,2,84,2,1,2,...,0.0,1,2,0,1,1,1,1,0,0
3,10,46,706.247579,14.791373,1,1,79,2,1,3,...,1.0,1,2,1,3,0,2,0,0,1
4,11,25,500.610860,2.146966,4,2,98,3,2,3,...,0.0,1,2,1,3,1,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1991,26,1437.994264,1.312387,3,1,84,2,1,3,...,0.0,0,2,1,3,1,2,1,0,0
796,1992,25,446.793690,6.906007,4,1,40,2,2,4,...,0.0,0,2,1,3,0,7,2,0,0
797,1993,26,97.635598,10.730256,3,3,37,3,1,3,...,0.0,0,1,1,3,1,1,2,0,0
798,1995,25,69.246616,0.878959,3,2,85,2,1,2,...,0.0,0,1,1,4,0,6,2,0,0


In [7]:
train = all_data.query('flg == 1')
test = all_data.query('flg == 0')

In [8]:
# trainデータの説明変数と目的変数をわける
linear_X_train = train.drop(['flg','Attrition'], axis=1)

# 目的変数はLinear,RandomForest共通
y_train = train['Attrition']
train_x, valid_x, train_y, valid_y = train_test_split(linear_X_train, y_train,test_size=0.2, random_state=0)
print(len(train_x))
print(len(valid_x))
print(len(train_y))
print(len(valid_y))
# testデータもlinear_X_trainと同じカラムにする
linear_test = test.drop(['Attrition','flg'],axis=1)

960
240
960
240


In [9]:
models = []
models.append(("KNC",KNeighborsClassifier()))
models.append(("DTC",DecisionTreeClassifier()))
models.append(("SVM",SVC()))
models.append(('lgb',lgb.LGBMClassifier()))
models.append(('clf',RandomForestClassifier()))
models.append(('extra_trees',ExtraTreesClassifier()))
models.append(('ada_boost',AdaBoostClassifier()))
models.append(('gradient_boosting',GradientBoostingClassifier()))
models.append(('xgboost',xgb.XGBClassifier()))

results = []
names = []
for name,model in models:
    kfold = KFold(n_splits=20, random_state=0, shuffle=True)
    result = cross_val_score(model,linear_X_train,y_train, cv = kfold, scoring = "accuracy")
    names.append(name)
    results.append(result)
for i in range(len(names)):
    print(names[i],results[i].mean())

c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 136, in __call__
    score = scorer._score(
  File "c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\utils\_response.py", line 85, in _get_response_values
    y_pred = prediction_method(X)
  File "c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_

KNC nan
DTC 0.7575000000000001
SVM 0.8224999999999998
lgb 0.8483333333333334
clf 0.8483333333333333
extra_trees 0.8450000000000001
ada_boost 0.8433333333333332
gradient_boosting 0.8491666666666667
xgboost 0.8491666666666665


In [10]:
svm = make_pipeline(StandardScaler(),
                        PCA(n_components=4),
                        SVC(),
                       )
svm.fit(train_x, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=4)), ('svc', SVC())])

In [11]:
oof =svm.predict(valid_x)
print('score', round(accuracy_score(valid_y, oof)*100,2))

score 81.17
